In [1]:
# Clone repo and go to repo's directory
!git clone https://github.com/kk-digital/kcg-ml-image-pipeline
!cd ./kcg-ml-image-pipeline

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Cloning into 'kcg-ml-image-pipeline'...
remote: Enumerating objects: 11244, done.
remote: Counting objects: 100% (1440/1440), done.
remote: Compressing objects: 100% (497/497), done.
Receiving objects:  20% (2249/11244), 9.42 MiB | 4.09 MiB/sReceiving objects:  32% (3645/11244), 12.19 MiB | 2.11 MiB/sReceiving objects:  32% (3646/11244), 13.29 MiB | 680.00 KiB/sfatal: the remote end hung up unexpectedly
fatal: early EOF
fatal: index-pack failed
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
/bin/bash: line 0: cd: ./kcg-ml-image-pipeline: No such file or directory


In [2]:
# Install requirements
!pip install minio torch==2.0.1 torchinfo==1.8.0

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [3]:
# Get minio client
MINIO_ADDRESS = "123.176.98.90:9000"
access_key = "GXvqLWtthELCaROPITOG"
secret_key = "DmlKgey5u0DnMHP30Vg7rkLT0NNbNIGaM8IwPckD"

import sys
import os
import io
sys.path.append(os.path.abspath('./kcg-ml-image-pipeline/'))
from training_worker.ab_ranking.model.ab_ranking_linear import ABRankingModel as ABRankingLinearModel
from utility.minio.cmd import is_minio_server_accessible, connect_to_minio_client, download_from_minio, get_file_from_minio

minio_client = connect_to_minio_client(MINIO_ADDRESS, access_key, secret_key)


Connecting to minio client...
Successfully connected to minio client...


In [4]:
import torch
# get linear positive embedding model
linear_model_path = "environmental/models/ranking/2023-12-08-00-score-linear-embedding-positive.pth"


model_file_data = get_file_from_minio(minio_client, 'datasets', linear_model_path)
if not model_file_data:
    raise Exception("No model file found at path: ", linear_model_path)

byte_buffer = io.BytesIO()
for data in model_file_data.stream(amt=8192):
    byte_buffer.write(data)
byte_buffer.seek(0)

linear_model = ABRankingLinearModel(inputs_shape=768)
model = torch.load(byte_buffer, map_location=torch.device('cpu'))
linear_model.model.load_state_dict(model['model_dict'])


<All keys matched successfully>

In [ ]:
# Get CLIP model
import math
import os
import time

import requests
from tqdm import tqdm

from configs import model_config
from utility.utils_logger import logger
from utility.labml.monit import section
from utility.minio.cmd import is_minio_server_accessible, connect_to_minio_client, download_from_minio

# create dirs
if not os.path.exists("input/model"):
  os.makedirs("input/model")
if not os.path.exists("./output"):
  os.makedirs("output")

config = model_config.ModelPathConfig()

def create_directory_tree_folders(config):
    config.create_paths()

with section("Creating directory tree folders."):
    create_directory_tree_folders(config)

logger.info("Downloading models. This may take a while.")

with section("Downloading CLIP model from minio"):
    # DOWNLOAD_BASE_CLIP_MODEL
    clip_path = "./kcg-ml-image-pipeline/input/model/clip/vit-large-patch14/model.safetensors"

    bucket_name = "models"
    object_name = "clip-vit-large-patch14/model.safetensors"
    download_from_minio(minio_client, bucket_name, object_name, clip_path)

!cp ./kcg-ml-image-pipeline/input/model/clip/vit-large-patch14/model.safetensors ./kcg-ml-image-pipeline/input/model/clip/txt_emb_model
!cp -r ./kcg-ml-image-pipeline/input ./

Creating directory tree folders....[DONE]	2.67ms                                  
INFO: Downloading models. This may take a while.
clip-vit-large-patch14/model.safetensors: |##################--| 1532.00 MB/1631.30 MB  93% [elapsed: 30:34 left: 01:58,  0.84 MB/sec]

In [ ]:
# Process CLIP model
from stable_diffusion.model.clip_text_embedder import CLIPTextEmbedder

# CLIP
text_embedder = CLIPTextEmbedder()
text_embedder.load_submodels()

In [ ]:
from utility.clip.clip_text_embedder import tensor_attention_pooling, tensor_max_pooling, tensor_max_abs_pooling

# For getting prompt score using linear embedding model
def get_prompt_linear_embedding_score(text_embedder, linear_model, prompt):
  # get embedding of prompt first
  embedding, _, attention_mask = text_embedder.forward_return_all(prompt)
  # average using attention mask
  average_pooled = tensor_attention_pooling(embedding, attention_mask)

  # use linear model
  score = linear_model.predict_positive_or_negative_only_pooled(average_pooled)

  return score.item()

In [ ]:
# Get phrase scores probability
filename = "2023_11_29_phrase_scores_probability_09_environmental_positive.csv"

download_urls = ['https://mega.nz/file/4I42VbTY#iZvzE8gc_vPbhvA8YEFqyIYToUoZqSJfvrRLksL4qq8']

# Destination paths for data transfer (can be in Google Drive or session storage)
destination_paths = ['./']

# Mega user name and password. Leave as empty string for anonymous login
# note: not used
mega_user_email = ''
mega_user_password = ''

In [ ]:
!pip install mega.py

In [ ]:
import os
from mega import Mega

# Create Mega User Object and Login
mega = Mega()
# mega_user = mega.login(mega_user_email, mega_user_password)


# Download files specified in download_urls to locations specified in destination_paths
for url, path in zip(download_urls, destination_paths):
    # Construct the full path to the file
    full_file_path = os.path.join(path, filename)

    # Check if file already exists
    if os.path.exists(full_file_path):
        print(f"File already exists: {full_file_path}")
    else:
        # Create directory if it does not exist
        os.makedirs(os.path.dirname(path), exist_ok=True)

        # Download file
        print(f"Downloading to {path}")
        mega.download_url(url, path)
        print(f"Download complete: {path}")


In [ ]:
import os

import numpy as np
import pandas as pd

import zipfile

from scipy import stats
from matplotlib import pyplot

In [ ]:
!stat 2023_11_29_phrase_scores_probability_09_environmental_positive.csv

# load prompt phrase scores

In [ ]:
#f = zipfile.ZipFile('./tmp/2023-12-04-phrase-scores-probability-environmental.zip')

#positive_phrase_df = pd.read_csv(f.open('2023_11_29_phrase_scores_probability_09_environmental_positive.csv'))
positive_phrase_df = pd.read_csv('2023_11_29_phrase_scores_probability_09_environmental_positive.csv')
positive_phrase_df

# histogram of score

In [ ]:
score = positive_phrase_df["score"].values

# negate the score
score = score * -1

In [ ]:
pyplot.figure(figsize=(12, 4))

pyplot.subplot(1, 2, 1)

_ = pyplot.hist(score, bins=1024, log=False)
pyplot.title('histogram of score')

pyplot.subplot(1, 2, 2)

_ = pyplot.hist(score, bins=1024, log=True)
pyplot.title('histogram of score (log scale)')

# normal test

In [ ]:
score_mean = np.mean(score)
score_std = np.std(score)

print("score_mean= ", score_mean)
print("score_std= ", score_std)

In [ ]:
stats.kstest(score, 'norm', args=(np.mean(score), np.std(score)))

# set probability formula

In [ ]:
boltzman_k = 1.0

def probablity_formula(score: np.ndarray, boltzman_temperature: float):

  p_array = np.exp(-(score / (boltzman_k * boltzman_temperature)))
  p_array_nomralized = p_array  / p_array.sum()  #np.linalg.norm(p_array )

  return p_array, p_array_nomralized

# trend of probability with temperature

In [ ]:
#raw outputs

boltzman_temperatures = list()
p_normalized_maxs = list()
p_normalized_mins = list()

for n in range(1,32):
  boltzman_temperature = float(n)

  p_array, p_array_nomralized = probablity_formula(score, boltzman_temperature)

  p_max = np.max(p_array)
  p_min = np.min(p_array)

  # print("n=%i, min_max_ratio= %.2f, probability_min= %.6f, probability_max= %.6f" % (n, p_max/p_min, p_min, p_max))

  p_normalized_max = np.max(p_array_nomralized)
  p_normalized_min = np.min(p_array_nomralized)

  # print("n=%i, normalized_min_max_ratio= %.2f, normalized_probability_min= %.6f, normalized_probability_max= %.6f" % (n, p_normalized_max/p_normalized_min, p_normalized_min, p_normalized_max))

  boltzman_temperatures.append(boltzman_temperature)
  p_normalized_maxs.append(p_normalized_max)
  p_normalized_mins.append(p_normalized_min)


pyplot.figure(figsize=(12, 4))

pyplot.subplot(1, 2, 1)

pyplot.plot(boltzman_temperatures, p_normalized_maxs, color='red', label='p_normalized_max')
pyplot.plot(boltzman_temperatures, p_normalized_mins, color='blue', label='p_normalized_min')
pyplot.legend()
pyplot.title(f'trend of normalized probability (linear scale)')

pyplot.subplot(1, 2, 2)

pyplot.semilogy(boltzman_temperatures, p_normalized_maxs, color='red', label='p_normalized_max')
pyplot.semilogy(boltzman_temperatures, p_normalized_mins, color='blue', label='p_normalized_min')
pyplot.legend()
pyplot.title(f'trend of normalized probability (log scale)')


Note:
- As temperature increases, selection probability goes to uniform
-

# specific temperature and analysis

In [ ]:
boltzman_temperature = 8

p_array, p_array_nomralized = probablity_formula(score, boltzman_temperature)

## histogram of normalized probability

In [ ]:
pyplot.figure(figsize=(12, 4))

pyplot.subplot(1, 2, 1)

_ = pyplot.hist(p_array_nomralized, bins=256, log=False)
pyplot.title(f'hist: normalized probability, temperature {boltzman_temperature} (linear scale)')

pyplot.subplot(1, 2, 2)

_ = pyplot.hist(p_array_nomralized, bins=256, log=True)
pyplot.title(f'hist: normalized probability, temperature {boltzman_temperature} (log scale)')

## histogram of cumulative probability

In [ ]:
def get_normalized_probability(scores,
                              boltzman_k,
                              boltzman_temperature):
  probability = np.exp(-(score / (boltzman_k * boltzman_temperature)))
  #normalized_probability = probability / np.linalg.norm(probability) #probability.sum()
  normalized_probability = probability / probability.sum()

  p_max = np.max(probability)
  p_min = np.min(probability)

  print("-----------------------------------------------")
  print("boltzman_k = ", boltzman_k)
  print("boltzman_temperature = ", boltzman_temperature)
  print("probability_max= ", p_max)
  print("probability_min= ", p_min)
  # _ = pyplot.hist(normalized_probability, bins=256, log=True)
  # pyplot.title(f'histogram of normalized probability, temperature {boltzman_temperature} (log scale)')

  return normalized_probability

In [ ]:
def get_cumulative_probability(normalized_probability):
  # we need the sorted indexes to get correct index of chosen phrase
  sorted_normalized_probability = []
  sorted_indexes = sorted(range(len(normalized_probability)), key=lambda x: normalized_probability[x],
                                  reverse=True)
  for i in sorted_indexes:
      sorted_normalized_probability.append(normalized_probability[i])
  sorted_normalized_probability = np.array(sorted_normalized_probability)

  # sorted_normalized_probability = np.array(sorted(normalized_probability, reverse=True))
  cumulative_probability = sorted_normalized_probability.cumsum()

  # assert cumulative_probability.sum() == 1.0, "Error: culmulative probability does not add to 1.0"

  return sorted_indexes, cumulative_probability

In [ ]:
boltzman_temperature = 2.0
normalized_probability = get_normalized_probability(score,
                                                    boltzman_k,
                                                    boltzman_temperature)
sorted_indexes, cumulative_probability = get_cumulative_probability(normalized_probability)


In [ ]:
### TODO:
### Assert than cumulative probability range is 0.00 to 1.00
#_ = pyplot.hist(cumulative_probability, bins=256, log=True)
#pyplot.title(f'histogram of cumulative probability, temperature {boltzman_temperature} (log scale)')

cumulative_probability_log2 = np.log2(cumulative_probability)

pyplot.title(f'CDF, T={boltzman_temperature} (log2 scale)')
pyplot.plot(cumulative_probability_log2, color='red', label='CDF_log2')
#pyplot.plot(boltzman_temperatures, p_normalized_mins, color='blue', label='p_normalized_min')
pyplot.legend()


Sampling

In [ ]:
#Add function
#uses cumulative probability distribution
#choose random number between 0.00 and 1.00
#use to choose next prompt from list
#keep adding phrases/prompts until token length would go over 75 (using token length parameter in csv)

#NOTE: make sure we dont use same prompt phrase twice in same generation

import math

# find the first element, whose cumulative prob is more than the random float
def find_first_element_binary_search(cumulative_prob_arr, random_float):
    low = 0
    high = len(cumulative_prob_arr) - 1
    mid = 0

    loop_count = 0
    while low < high:
        loop_count += 1
        assert loop_count < 32, "Error: binary search loop count is more than 32"

        mid = (high + low) / 2
        mid = math.floor(mid)

        # If random_float is greater, ignore left half
        if cumulative_prob_arr[mid] < random_float:
            low = mid + 1
        # If random_float is smaller, ignore right half
        elif cumulative_prob_arr[mid] >= random_float:
            high = mid - 1

        # use this index since sometimes the exact
        # random num is not in the list
        if low == high:
            # assert cumulative_prob_arr[low-1] < random_float
            # assert cumulative_prob_arr[low] >= random_float, "{} >= {}, next index val={}".format(cumulative_prob_arr[low], random_float, cumulative_prob_arr[low+1])
            # assert round(cumulative_prob_arr[low], 4) >= 0.0, "val={}".format(cumulative_prob_arr[low])
            # assert round(cumulative_prob_arr[low], 4) <= 1.0, "val={}".format(cumulative_prob_arr[low])

            return low


    # If we reach here, then the element was not present
    return -1

In [ ]:
import random

def generate_prompt(positive_phrase_df,
                    positive_phrase_origin_indexes,
                    positive_cumulative_probability_arr,
                    ):
    max_token_size = 75
    comma_token_size = 1

    positive_prompt_total_token_size = 0
    positive_prompt = []
    positive_prompt_indexes = []
    positive_used_phrase_dict = {}

    positive_cumulative_probability_arr_min = positive_cumulative_probability_arr.min()
    positive_cumulative_probability_arr_max = positive_cumulative_probability_arr.max()
    # positive prompt
    while positive_prompt_total_token_size < max_token_size:
        random_float = random.uniform(positive_cumulative_probability_arr_min,
                                      positive_cumulative_probability_arr_max)
        random_index = find_first_element_binary_search(positive_cumulative_probability_arr, random_float)
        if random_index in positive_used_phrase_dict:
            continue

        prompt_index = positive_phrase_origin_indexes[random_index]
        random_phrase = positive_phrase_df["phrase"][prompt_index]

        chosen_phrase_size = positive_phrase_df["token length"][prompt_index]
        sum_token_size = positive_prompt_total_token_size + chosen_phrase_size + comma_token_size
        if sum_token_size < max_token_size:
            # update used array
            positive_used_phrase_dict[random_index] = 1
            positive_prompt.append(str(random_phrase))
            positive_prompt_indexes.append(prompt_index)
            positive_prompt_total_token_size = sum_token_size
        else:
            break

    positive_prompt_str = ', '.join([prompt for prompt in positive_prompt])

    return positive_prompt_str, positive_prompt_indexes

In [ ]:
def get_variance(energies, mean):
  energies = np.array(energies)
  diff = energies - mean
  product = diff * diff
  sum = product.sum()
  variance = sum/len(energies)

  return variance

In [ ]:
def get_prompt_score(phrase_scores_arr, prompt_indexes):
  score = 0
  for index in prompt_indexes:
    score += phrase_scores_arr[index]

  return score

Test generate prompts

In [ ]:
# generate N prompts
prompts = []
prompt_indexes_arr = []
prompt_count = 5
for i in range(prompt_count):
  prompt, prompt_indexes = generate_prompt(positive_phrase_df,
                                           sorted_indexes,
                                           cumulative_probability)
  prompts.append(prompt)
  prompt_indexes_arr.append(prompt_indexes)

for i in range(len(prompts)):
  #Then add up "score" for each prompt phrase, to get the total score estimate for whole prompt
  prompt_score = get_prompt_score(score, prompt_indexes_arr[i])
  print("[{}]: Score = {}, Prompt = {}".format(i, prompt_score, prompts[i]))



Graph average prompt energy per temperature

In [ ]:
from tqdm import tqdm
# generate N prompts
prompt_count = 1024

average_linear_embedding_scores_per_temp = []
average_energy_per_temp = []
variance_per_temp = []
temperature_range = [i for i in range(4,33)]
print("temperature_range=", temperature_range)
for temperature in tqdm(temperature_range):
  prompt_indexes_arr = []
  prompt_str_arr = []
  normalized_probability = get_normalized_probability(score,
                                                      boltzman_k,
                                                      temperature)
  sorted_indexes, cumulative_probability = get_cumulative_probability(normalized_probability)

  # generate prompt
  for i in range(prompt_count):
    prompt, prompt_indexes = generate_prompt(positive_phrase_df,
                                            sorted_indexes,
                                            cumulative_probability)
    prompt_indexes_arr.append(prompt_indexes)
    prompt_str_arr.append(prompt)

  prompt_energies = []
  prompt_linear_embedding_scores = []
  for i in range(len(prompt_indexes_arr)):
    #Then add up "score" for each prompt phrase, to get the total score estimate for whole prompt
    prompt_energy = get_prompt_score(score, prompt_indexes_arr[i])
    prompt_energies.append(prompt_energy)

    # get linear embedding score
    linear_embedding_score = get_prompt_linear_embedding_score(text_embedder,
                                                               linear_model,
                                                               prompt_str_arr[i])
    prompt_linear_embedding_scores.append(linear_embedding_score)

  average_energy = np.mean(prompt_energies)
  variance = get_variance(prompt_energies, average_energy)
  average_linear_embedding_score = np.mean(prompt_linear_embedding_scores)
  print("mean energy = ", average_energy)
  print("variance = ", variance)
  print("linear embedding mean score = ", average_linear_embedding_score)

  average_energy_per_temp.append(average_energy)
  variance_per_temp.append(variance)
  average_linear_embedding_scores_per_temp.append(average_linear_embedding_score)

pyplot.figure(figsize=(16, 4))
# mean energy
pyplot.subplot(1, 2, 1)
_ = pyplot.plot(average_energy_per_temp, color='red', label='Average Energy', zorder=0)
pyplot.title(f'Average Prompt Energy vs Temperature')
pyplot.xlabel("Temperature")
pyplot.ylabel("Average Prompt Energy")
pyplot.xticks([i for i in range(len(temperature_range))], labels=temperature_range)
pyplot.xticks(rotation=90)
pyplot.legend()

# variance
pyplot.subplot(1, 2, 2)
_ = pyplot.plot(variance_per_temp, color='red', label='Variance', zorder=0)
pyplot.title(f'Variance vs Temperature')
pyplot.xlabel("Temperature")
pyplot.ylabel("Variance")
pyplot.xticks([i for i in range(len(temperature_range))], labels=temperature_range)
pyplot.xticks(rotation=90)
pyplot.legend()

# linear embedding mean energy
pyplot.subplot(1, 2, 3)
_ = pyplot.plot(average_linear_embedding_scores_per_temp, color='red', label='Linear Embedding Average Energy', zorder=0)
pyplot.title(f'Linear Embedding Average Prompt Energy vs Temperature')
pyplot.xlabel("Temperature")
pyplot.ylabel("Linear Embedding Average Prompt Energy")
pyplot.xticks([i for i in range(len(temperature_range))], labels=temperature_range)
pyplot.xticks(rotation=90)
pyplot.legend()

TODO: Max Entropy Sampling
- problem: assign probabilites
- solution:
-- assign probabilites so that that
- sum(P(x)*score(x)) = EnergyTarget
- and subject to MAX sum(p(x)*ln(P(x)); maximum entropy
-- example: choose probabilities so that they average to EnergyTarget

Then sample from the probability distribution sequentially

